<a href="https://colab.research.google.com/github/FrederiKob/Kelly_Replication_Code/blob/main/Run_Rudimentary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use git to make clone of GitHub repository

In [33]:
!git clone https://github.com/FrederiKob/Kelly_Replication_Code.git

Cloning into 'Kelly_Replication_Code'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 55 (delta 30), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (55/55), 475.80 KiB | 12.20 MiB/s, done.
Resolving deltas: 100% (30/30), done.


Draw Data from Rudimentary_Replication_Kelly_DataPrep Script

In [35]:
%run "Rudimentary_Replication_Kelly_DataPrep.ipynb"

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Import Library_Function by running script Library_Function

In [37]:
%run "Library_Function.ipynb"

# Begin Rudimentary Run

Define Parameters for a restricted and rudimentary run-through of the general model script

*   Restrict predictions to $P = 12000$ and $alpha = 1000$
*   begin and end are reduced to a few years to get a good gist of the quality of the model/similarities to Kelly et al.
*   Furthermore, faster run-through, given the reduced computational demand



In [ ]:
P = 12000
alpha = 1000

# 50 should be sufficient to get a grasp of the results
max_iterations = 50

# begin and end include very volatile period (better portrayal of bheaviour of market timing positions)
begin = "1940-01-01"
end = "1950-01-01"
# determine the according index location
idx_begin = np.where(ret_std.index == begin)[0][0]
idx_end = np.where(ret_std.index == end)[0][0]

# reduce ret_std and pred_std so that data-load is reduced
ret_std = ret_std.iloc[:idx_end+1]
pred_std = pred_std.iloc[:idx_end+1,:]

Run Ridge Regression over loops (very sloppy but rudimentary)

In [ ]:
from sklearn.linear_model import Ridge

res1,res2,res3 = [],[],[]
for seed in range(max_iterations):
    pred1,pred2,pred3 = [],[],[]
    signals = generate_Signals(pred_std=pred_std, P=P, use_seed=seed, w_i_all=w_i_all)

    for t in range(idx_begin,idx_end):
        x1,y1 = TEST_generate_X_y(signals=signals, ret_std=ret_std, idx_start=t, T=12, P=P, vol_stand=True)
        clf1 = Ridge(alpha=alpha, fit_intercept=False)
        clf1.fit(X=x1[0],y=y1[0])
        pred1.append(clf1.predict(x1[1])[0])

        x2,y2 = TEST_generate_X_y(signals=signals, ret_std=ret_std, idx_start=t, T=60, P=P, vol_stand=True)
        clf2 = Ridge(alpha=alpha, fit_intercept=False)
        clf2.fit(X=x2[0],y=y2[0])
        pred2.append(clf2.predict(x2[1])[0])

        x3,y3 = TEST_generate_X_y(signals=signals, ret_std=ret_std, idx_start=t, T=120, P=P, vol_stand=True)
        clf3 = Ridge(alpha=alpha, fit_intercept=False)
        clf3.fit(X=x3[0],y=y3[0])
        pred3.append(clf3.predict(x3[1])[0])

    print("Iteration number {} out of {} is done!".format(seed,max_iterations))
    res1.append(pred1)
    res2.append(pred2)
    res3.append(pred3)